In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
json_response = requests.get('http://localhost:8501/v1/models/animals/versions/1/metadata')
json_response.json()

{'error': 'Servable not found for request: Specific(animals, 1)'}

In [8]:
# Check which versions are available
import subprocess

# Get container ID
result = subprocess.run(['docker', 'ps', '--filter', 'ancestor=tensorflow/serving', '--format', '{{.ID}}'], 
                       capture_output=True, text=True)
container_id = result.stdout.strip()
print(f"Container ID: {container_id}")

# Check what's in the models directory inside the container
result = subprocess.run(['docker', 'exec', container_id, 'ls', '-la', '/models/animals/'], 
                       capture_output=True, text=True)
print("\n/models/animals/ contents:")
print(result.stdout)

# Check model metadata endpoint
import requests
response = requests.get('http://localhost:8501/v1/models/animals')
print("\nModel status:")
print(response.json())

Container ID: 14fe19427eb8

/models/animals/ contents:
total 16
drwxrwxrwx+ 4 1000 1000 4096 Nov 24 18:55 .
drwxrwxrwx+ 3 1000 1000 4096 Nov 24 19:43 ..
drwxrwxrwx+ 4 1000 1000 4096 Nov 24 19:32 1
drwxr-xr-x+ 4 1000 1000 4096 Nov 24 18:56 2


Model status:
{'model_version_status': [{'version': '2', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}, {'version': '1', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}]}


In [7]:
# Check if saved_model.pb exists and its size for both versions
result = subprocess.run(['docker', 'exec', container_id, 'find', '/models/animals/', '-name', 'saved_model.pb', '-ls'], 
                       capture_output=True, text=True)
print("saved_model.pb files:")
print(result.stdout)

# Check TensorFlow Serving logs for errors
result = subprocess.run(['docker', 'logs', '--tail', '100', container_id], 
                       capture_output=True, text=True)
print("\nRecent logs (errors related to version 1):")
for line in result.stderr.split('\n'):
    if 'animals' in line.lower() or 'error' in line.lower() or 'fail' in line.lower():
        print(line)

saved_model.pb files:


Recent logs (errors related to version 1):
Error response from daemon: No such container: 966f6b59d9b5


In [9]:
# Verify both versions are available
response = requests.get('http://localhost:8501/v1/models/animals')
print("Available model versions:")
for version_status in response.json()['model_version_status']:
    print(f"  Version {version_status['version']}: {version_status['state']}")

Available model versions:
  Version 2: AVAILABLE
  Version 1: AVAILABLE


In [ ]:
# Test model version 2 inference with proper image preprocessing
import numpy as np
from PIL import Image
import requests
import io

# Create a test image (128x128 for version 2)
test_image = np.random.randint(0, 255, (128, 128, 3), dtype=np.uint8)
img = Image.fromarray(test_image)

img_array = np.array(img, dtype=np.float32)
img_array = img_array / 255.0  # Normalize to [0, 1]
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

payload = {
    "instances": img_array.tolist()
}

# Send prediction request
url = "http://localhost:8501/v1/models/animals/versions/2:predict"
response = requests.post(url, json=payload)

print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    result = response.json()
    print(f"Predictions: {result['predictions']}")
else:
    print(f"Error: {response.text}")

Status Code: 200
Predictions: [[0.843864381]]


In [ ]:
# Test model version 1 inference with proper image preprocessing
import numpy as np
from PIL import Image
import requests
import io

# Create a test image (150x150 for version 1)
test_image = np.random.randint(0, 255, (150, 150, 3), dtype=np.uint8)
img = Image.fromarray(test_image)

img_array = np.array(img, dtype=np.float32)
img_array = img_array / 255.0  # Normalize to [0, 1]
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
payload = {
    "instances": img_array.tolist()
}

# Send prediction request
url = "http://localhost:8501/v1/models/animals/versions/1:predict"
response = requests.post(url, json=payload)

print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    result = response.json()
    print(f"Predictions: {result['predictions']}")
else:
    print(f"Error: {response.text}")

Status Code: 200
Predictions: [[0.290577501, 0.443411142, 0.266011298]]
